# OpenAI API 基本使用

关键参数：

- model，使用哪个模型，这些就填入哪个模型的名称
- messages，发给模型的消息，这里的消息是一个消息列表，可以简单地把它理解成一个历史的消息列表，这样模型就知道了上下文信息。
    - 模型记忆的本质，就是这里的历史消息列表，让模型有记忆，就是按下面对 message 格式的介绍，把用户（user）和 AI(assistant) 的对话信息放到历史消息列表总
    - 列表中的每一项，都是一个 message，每条消息通常包括两个部分：角色（role）和内容（content）
    - role 常用的有三个：`system`, `user`和`assistant`
        - system，给 AI 设定的身份、背景信息等，通常放到 system 信息中
        - user，我们发给 AI 的信息，放到 user 信息中
        - assistant，AI 给我们的回复信息，放到 assistant 信息中
- n，控制 AI 给我们生成回复的个数
- temperature，用来设定大模型回复的确定性，值越小，表示确定性越强，值越大，表示随机性越强，取值范围[0, 2], 通常取值 0.8
- max_tokens，控制返回内容的最大 token 数量

另外，还有一个与 temperature 作用类似的参数：`top_p`，也是用来设定大模型回复的确定性，值越小，表示确定性越强，值越大，表示随机性越强，取值范围[0, 2]，它与`temperature` 只有填入一个即可



In [1]:
from openai import OpenAI

client = OpenAI()

model_name = "openai/gpt-4o-mini"
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system",
         "content": "You are an IT expert. You can help me by answering my questions. You can also ask me questions."},
        {"role": "user", "content": "用一句话介绍微服务架构。"},
    ],
    n=1,
    temperature=0.8,
    max_tokens=256
)

print(response.choices[0].message.content)

微服务架构是一种将应用程序拆分为一组小型、独立的服务，每个服务负责特定功能，通过轻量级的通信协议进行交互，从而提高系统的灵活性和可维护性。


# OpenAI API 返回数据的结构化输出

OpenAI 的 completion 接口，可以通过`response_format` 参数，指定返回数据的格式。

以往该参数的输入值为: `{ "type": "json_object" }`，即 “JSON mode”，该方式需要配合提示词工程，在提示词中约定好输出 json 数据的格式，但不能严格保证返回数据一定按提示词的 json 格式生成。

现在，OpenAI 较新的模型开始支持 `Structured Outputs` 形式的输出，即不需借助提示词，直接保证大模型严格按照约定的 Json 格式进行输出，详见官方文档：[Structured Outputs](https://platform.openai.com/docs/guides/structured-outputs/introduction)


支持 Structured Outputs 模型, 从 GPT-4o 开始，具体为:

- gpt-4o-mini-2024-07-18 and later
- gpt-4o-2024-08-06 and later


注意， `Structured Outputs` 对 python 和 openai 版本均有要求，使用的方法也不同：

- python >= 3.12.4
- openai >=1.40.0
- 使用 `client.beta.chat.completions.parse()`方法



In [7]:
# Structured Outputs Example

from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]


model_name = "openai/gpt-4o-mini"
msgs = [
    {"role": "system", "content": "Extract the event information."},
    {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
]

# 原来的写法：
# response = client.chat.completions.create(
#     model=model_name,
#     messages=msgs,
#     temperature=0.8,
#     n=1,
# )
# msg = response.choices[0].message
# print("result: ", msg.content)

# Structured Outputs
response = client.beta.chat.completions.parse(
    model=model_name,
    messages=msgs,
    temperature=0.8,
    n=1,
    response_format=CalendarEvent
)

msg = response.choices[0].message
print("content:", msg.content, "\ntype: ", type(msg.content))

parsed = msg.parsed
print("parsed object: ", parsed, "\ntype: ", type(parsed))

content: {"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]} 
type:  <class 'str'>
parsed object:  name='Science Fair' date='Friday' participants=['Alice', 'Bob'] 
type:  <class '__main__.CalendarEvent'>


In [23]:
# JSON mode Example

from openai import OpenAI

client = OpenAI()

model_name = "openai/gpt-4o-mini"
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "system",
            "content": "You extract email addresses into JSON data."
        },
        {
            "role": "user",
            "content": "Feeling stuck? Send a message to help@mycompany.com."
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "email_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "email": {
                        "description": "The email address that appears in the input",
                        "type": "string"
                    },
                    "additionalProperties": False
                }
            }
        }
    }
)

print(response.choices[0].message.content);

{"email":"help@mycompany.com"}


In [24]:
# JSON mode Example 2

from openai import OpenAI

client = OpenAI()

model_name = "openai/gpt-4o-mini"
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "system",
            "content": """You extract email addresses into JSON data, output should be a json string.
            For example, as for abc@email.com, your output should be: '{"email":"help@mycompany.com"}'.
            Constraints: you should output json string only, do not use markdown format"""
        },
        {
            "role": "user",
            "content": "Feeling stuck? Send a message to help@mycompany.com."
        }
    ],
)

print(response.choices[0].message.content);

{"email":"help@mycompany.com"}


In [8]:
# Structured Outputs -- Chain of Thought Example

from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()


class Step(BaseModel):
    explanation: str
    output: str


class MathReasoning(BaseModel):
    steps: list[Step]
    final_output: str


model_name = "openai/gpt-4o-mini"
response = client.beta.chat.completions.parse(
    model=model_name,
    messages=[
        {"role": "system",
         "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -25"}
    ],
    response_format=MathReasoning
)

print(response.choices[0].message.content)
print(response.choices[0].message.parsed)


{"steps":[{"explanation":"To isolate the term with x, start by subtracting 7 from both sides of the equation.","output":"8x + 7 - 7 = -25 - 7"},{"explanation":"Simplifying both sides gives you: 8x = -32.","output":"8x = -32"},{"explanation":"Next, to solve for x, divide both sides of the equation by 8.","output":"x = -32 / 8"},{"explanation":"Calculating the division gives you: x = -4.","output":"x = -4"}],"final_output":"The solution to the equation 8x + 7 = -25 is x = -4."}
steps=[Step(explanation='To isolate the term with x, start by subtracting 7 from both sides of the equation.', output='8x + 7 - 7 = -25 - 7'), Step(explanation='Simplifying both sides gives you: 8x = -32.', output='8x = -32'), Step(explanation='Next, to solve for x, divide both sides of the equation by 8.', output='x = -32 / 8'), Step(explanation='Calculating the division gives you: x = -4.', output='x = -4')] final_output='The solution to the equation 8x + 7 = -25 is x = -4.'


In [13]:
# 内容审查

from enum import Enum
from typing import Optional
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()


class Category(str, Enum):
    violence = "violence"
    sexual = "sexual"
    self_harm = "self_harm"


class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violence: Optional[str]


model_name = "openai/gpt-4o-mini"
response = client.beta.chat.completions.parse(
    model=model_name,
    messages=[
        {"role": "system",
         "content": "Determine if the user input violates specific guidelines and explain if they do."},
        {"role": "user", "content": "How do I prepare for a job interview?"}
    ],
    response_format=ContentCompliance,
)

print(response.choices[0].message.content)

{"is_violating":false,"category":null,"explanation_if_violence":null}


# 大模型的记忆

大模型记忆的本质，就是把与大模型的对话历史，也放到 messages 里去，一并发给大模型，这样在本次的对话中，大模型就知道之前都说了什么。


In [6]:
from openai import OpenAI

client = OpenAI()
model_name = "openai/gpt-4o-mini"

msgs = [
    {"role": "system", "content": "you are a helpful assistant"}
]

def make_message(role, content) -> dict:
    return {"role": role, "content": content}

while True:
    user_input = input("You:> ")
    if user_input.lower() == "exit":
        break

    print("You:> ", user_input)
    msg = make_message("user", user_input)
    msgs.append(msg)

    resp = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "you are a helpful assistant"},
            {"role": "user", "content": user_input},
        ],
        temperature=0.8,
        max_tokens=256
    )
    assistant_msg = resp.choices[0].message.content
    print("AI: ", assistant_msg)



You:>  我周五去看电影
AI:  那听起来很不错！你打算看什么电影呢？或者你已经选好影院了吗？
You:>  我刚才说周五去干什么？
AI:  抱歉，我无法查看之前的对话内容。请告诉我你提到的周五要做什么，我会尽力帮助你！


In [5]:
from openai import OpenAI

client = OpenAI()
model_name = "openai/gpt-4o-mini"

msgs = [
    {"role": "system", "content": "you are a helpful assistant"}
]

def make_message(role, content) -> dict:
    return {"role": role, "content": content}

while True:
    user_input = input("You:> ")
    if user_input.lower() == "exit":
        break

    print("You:> ", user_input)
    msg = make_message("user", user_input)
    msgs.append(msg)

    resp = client.chat.completions.create(
        model=model_name,
        messages=msgs,
        temperature=0.8,
        max_tokens=256
    )
    assistant_msg = resp.choices[0].message.content
    print("AI: ", assistant_msg)
    msg = make_message("assistant", assistant_msg)
    msgs.append(msg)



You:>  我周五晚上去看电影
AI:  听起来不错！你要看什么电影呢？有没有特别期待的影片或者类型？
You:>  我周五晚上去干什么？
AI:  你提到周五晚上去看电影，所以可以说你计划去看电影。如果你有其他安排或者想做的事情，也可以分享出来！


# 流式输出

In [18]:
from openai import OpenAI

client = OpenAI()
msgs = [
    {"role": "system",
     "content": "我希望你充当 IT 专家。我会向您提供有关我的技术问题所需的所有信息，而您的职责是解决我的问题。你应该使用你的项目管理知识，敏捷开发知识来解决我的问题。在您的回答中使用适合所有级别的人的智能、简单和易于理解的语言将很有帮助。用要点逐步解释您的解决方案很有帮助。我希望您回复解决方案，而不是写任何解释。"},
    {"role": "user", "content": "解释微服务架构"}
]
model_name = "openai/gpt-4o-mini"
stream = client.chat.completions.create(
    model=model_name,
    messages=msgs,
    n=1,
    stream=True
)

for chunk in stream:
    if chunk.choices[0].delta.content is None:
        break
    # print(chunk.choices[0].delta.content, end="")
    print(chunk.choices[0].delta.content, end="/")

/微/服务/架/构/是一/种/软件/架/构/风/格/，/旨/在/将/单/一/应用/程序/划/分/为/一/组/小/的/、/独/立/的/服务/。/以下/是/微/服务/架/构/的/主要/特点/和/优势/：

/1/./ **/服务/独/立/性/**/：
/  / -/ 每/个/微/服务/都是/一个/独/立/的/功能/模块/，可以/独/立/开发/、/部署/和/扩/展/。
/  
/2/./ **/技术/多/样/性/**/：
/  / -/ 不/同/的/服务/可以/使用/不同/的/编/程/语言/、/框/架/和/数据库/。
/  
/3/./ **/容/错/性/**/：
/  / -/ 单/个/服务/的/故/障/不会/影响/整/个平台/，其/它/服务/仍/可以/继续/运行/。
/  
/4/./ **/持续/交/付/与/部署/**/：
/  / -/ 支/持/快速/迭/代/，/能够/实现/频/繁/的/更新/和/发布/。

/5/./ **/团队/自治/**/：
/  / -/ 每/个/团队/可以/对/其/服务/负责/，从/而/提升/开发/效率/和/推动/更/快/的/决/策/。

/6/./ **/API/ /交/互/**/：
/  / -/ 服务/之间/通过/轻/量/级/的/通信/机制/（/通常/是/ HTTP///REST/ 或/消息/队/列/）/进行/交/互/。

/7/./ **/易/于/扩/展/**/：
/  / -/ 可以/根据/服务/的/需求/独/立/扩/展/某/些/服务/，而/不是/整体/扩/展/整个/应用/。

/8/./ **/监/控/与/管理/**/：
/  / -/ /需要/有效/的/监/控/和/管理/工具/，以/处理/服务/间/的/复杂/性/和/可/追/踪/性/。

/**/总结/**/：/微/服务/架/构/通过/将/应用/程序/拆/分/为/小/的/独/立/服务/，/提供/了/灵/活/性/、/可/扩/展/性/和/恢复/力/，/适/合/现代/云/原/生/开发/。///

# Function calling




In [7]:
from openai import OpenAI

client = OpenAI()


# define a tool
def get_current_cluster_state(cluster_name):
    print(f"cluster:{cluster_name}")
    return """ERROR: Failed to pull image "chaocai/docker/dsp:latest"""


# tool description for LLM
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_cluster_state",
            "description": "Get the current state in a given cluster",
            "parameters": {
                "type": "object",
                "properties": {
                    "cluster_name": {
                        "type": "string",
                        "description": "the name of the cluster"
                    },
                },
                "required": ["cluster_name"],  # 配置必填参数
                "additionalProperties": False,
            },
        },
    }
]

model_name = "openai/gpt-4o-mini"
response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": "What's wrong with the cluster 'DSP'?"}],
    tools=tools,
    tool_choice="auto",  # `auto` is the default if tools are present
)

# 执行 function call 时，tool_calls 有值，且 content 中没有值
assert response.choices[0].message.tool_calls is not None
assert response.choices[0].message.content is None
assert len(response.choices[0].message.tool_calls) > 0


In [8]:
tool_calls = response.choices[0].message.tool_calls
print(len(tool_calls))
print(tool_calls[0].function.name)  # 函数名
print(tool_calls[0].function.arguments)  # 函数的参数和值，json 字符串格式

1
get_current_cluster_state
{"cluster_name":"DSP"}


In [9]:
print(response.choices[0].message.role)
print(response.choices[0].message)

assistant
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Ps8ZNpWtV47pUXkNLobU0prs', function=Function(arguments='{"cluster_name":"DSP"}', name='get_current_cluster_state'), type='function', index=0)])


In [3]:
# 根据 function 结果进行函数调用

import json

tools_map = {
    "get_current_cluster_state": get_current_cluster_state
}

tool_name = tool_calls[0].function.name
args = json.loads(tool_calls[0].function.arguments)

tool = tools_map[tool_name]

res = tool(**args)
print(res)

cluster:DSP
ERROR: Failed to pull image "chaocai/docker/dsp:latest


In [4]:
# 当大模型判断不需要进行 function call 时，会自己回答问题

model_name = "openai/gpt-4o-mini"
response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": "你是谁?"}],
    tools=tools,
    tool_choice="auto",  # `auto` is the default if tools are present
)

# 不执行 function call 时，tool_calls 没有值，且 content 中有值
assert response.choices[0].message.tool_calls is None
assert response.choices[0].message.content is not None

print(response.choices[0].message.content)

我是一个人工智能助手，旨在回答你的问题、提供信息和帮助你解决各种问题。有什么我可以为你做的吗？


## Function call 进阶用法

当我们让 LLM 执行完 Function call 后，除了调用工具执行结果外，有时我们还希望再把工具调用完的结果反馈给 LLM，让大模型做进一步判断后续的执行。

实现这个目标，本质是把 function call 的结果加到 LLM 的记忆中去，这里的记忆和文本内容的记忆有些许差别，详情如下：

In [18]:
# Function call + 记忆

import json
from openai import OpenAI
from openai.types.chat import ChatCompletionMessage

client = OpenAI()
model_name = "openai/gpt-4o-mini"


def chat(messages: list) -> ChatCompletionMessage:
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tool_choice="auto",
        tools=tools,
    )
    msg = response.choices[0].message

    if not msg.tool_calls:
        return msg

    tool_call = msg.tool_calls[0]
    func_name = tool_call.function.name
    func_args = tool_call.function.arguments

    res = call_tool(func_name, func_args)
    messages.append({
        "role": msg.role,
        # deprecated
        "function_call": {
            "name": func_name,
            "arguments": func_args,
        },
        "content": None
    })

    messages.append({
        "role": "function",
        "name": func_name,
        "content": res,
    })

    return chat(messages)

def chat2(messages: list) -> ChatCompletionMessage:
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tool_choice="auto",
        tools=tools,
    )
    msg = response.choices[0].message

    if not msg.tool_calls:
        return msg

    tool_call = msg.tool_calls[0]
    func_name = tool_call.function.name
    func_args = tool_call.function.arguments

    res = call_tool(func_name, func_args)
    messages.append({
        "role": msg.role,
        # deprecated
        "function_call": {
            "name": func_name,
            "arguments": func_args,
        },
        "content": None
    })

    messages.append({
        "role": "tool",
        "name": tool_call.id,
        "content": res,
    })

    return chat(messages)

def chat3(messages: list) -> ChatCompletionMessage:
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tool_choice="auto",
        tools=tools,
    )
    msg = response.choices[0].message

    if not msg.tool_calls:
        return msg

    tool_call = msg.tool_calls[0]
    func_name = tool_call.function.name
    func_args = tool_call.function.arguments

    res = call_tool(func_name, func_args)
    messages.append({
        "role": msg.role,
        "tool_calls": [{
            "id": tool_call.id,
            "function": tool_call.function,
            "type": "function"
        }],
        "content": None
    })

    messages.append({
        "role": "tool",
        "name": tool_call.id,
        "content": res,
    })

    return chat(messages)


def call_tool(tool_name: str, tool_args: str):
    tool = tools_map[tool_name]
    args = json.loads(tool_args)

    return tool(**args)

def run(inputs):
    msg=[{"role":"user","content":inputs}]
    res = chat3(msg)
    return res.content

In [12]:
res = run("hello")
print(res)

Hello! How can I assist you today?


In [22]:
result = run("What's wrong with the cluster 'DSP'? ")
print(result)

cluster:DSP
The cluster 'DSP' is encountering an error because it failed to pull the image "chaocai/docker/dsp:latest". This could be due to various reasons such as the image not being available, network issues, or permission problems.


In [19]:
result = run("What's wrong with the cluster 'DSP'? And if there's an error, give me some suggestion. ")
print(result)

cluster:DSP
The current issue with the cluster 'DSP' is that it failed to pull the Docker image "chaocai/docker/dsp:latest." This type of error can happen for several reasons. Here are some suggestions to resolve the issue:

1. **Check Internet Connectivity**: Ensure that the cluster has access to the internet, as it needs to connect to the Docker registry to pull the image.

2. **Image Availability**: Verify if the image "chaocai/docker/dsp:latest" exists in the Docker registry. You can do this by visiting Docker Hub or the relevant registry where the image is supposed to be hosted.

3. **Authentication Issues**: If the image is in a private repository, ensure that the correct authentication credentials are provided.

4. **Check for Typos**: Double-check the image name and tag for any typos.

5. **Resource Limits**: Ensure that the cluster has enough resources (CPU, memory, etc.) to pull and run the image.

6. **Docker Daemon Status**: Ensure that the Docker daemon is running properly